In [ ]:
# =====================================================
# Cell 1: Load All Results
# =====================================================
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import json

# Scenario definitions
scenarios = ['low', 'medium', 'high']
velocities = {'low': 0.0, 'medium': 0.1, 'high': 1.0}
colors = {'low': 'green', 'medium': 'blue', 'high': 'red'}
labels = {'low': 'v=0 m/d (no flow)', 'medium': 'v=0.1 m/d', 'high': 'v=1.0 m/d'}

# Load results
results = {}
workspace = Path('../workspace')

for scenario in scenarios:
    json_path = workspace / f'modflow_gwflow_{scenario}_results.json'
    if json_path.exists():
        with open(json_path, 'r') as f:
            results[scenario] = json.load(f)
        print(f"Loaded {scenario}: v={results[scenario]['velocity_md']} m/d, MAE={results[scenario]['mae_vs_eed']:.3f}°C")
    else:
        print(f"WARNING: {scenario} results not found at {json_path}")

print(f"\nLoaded {len(results)} of 3 scenarios")

In [ ]:
# =====================================================
# Cell 2: EED Reference Data
# =====================================================

# EED reference data (JAN-DEC, 25 years)
eed_base = np.array([10.5, 10.6, 11.3, 12.3, 13.0, 15.4, 17.9, 18.2, 13.9, 12.7, 12.0, 10.9])
n_yr = 25
eed_25yr = np.tile(eed_base, n_yr)

# Peak values
eed_peak_heat_jan = 6.91
eed_peak_cool_aug = 22.4

# X-axis for plotting
x_years = np.arange(len(eed_25yr)) / 12.0

print(f"EED reference: {len(eed_25yr)} months, {n_yr} years")

In [ ]:
# =====================================================
# Cell 3: Comprehensive Comparison Plot (25 Years)
# =====================================================

fig, ax = plt.subplots(figsize=(16, 8))
ax.set_facecolor('#e8e8ff')

# Plot EED reference
ax.plot(x_years, eed_25yr, color='gray', ls='--', lw=2, alpha=0.8, label='EED (reference)')

# Plot each scenario
for scenario in scenarios:
    if scenario in results:
        Tf_jan = np.array(results[scenario]['Tf_jan'])
        ax.plot(x_years[:len(Tf_jan)], Tf_jan, 
                color=colors[scenario], lw=1.5, 
                label=f"{labels[scenario]} (MAE={results[scenario]['mae_vs_eed']:.2f}°C)")

ax.set_xlabel('Years', fontsize=12)
ax.set_ylabel('Fluid temperature [°C]', fontsize=12)
ax.set_title('MODFLOW Groundwater Flow Sensitivity Analysis\n25-Year Comparison', fontsize=14, fontweight='bold')
ax.set_xlim(0, 25)
ax.set_ylim(9, 20)
ax.set_xticks(range(0, 26, 2))
ax.grid(True, alpha=0.3)
ax.legend(loc='upper right', fontsize=11)

plt.tight_layout()
plt.savefig('../figures/modflow_gwflow_comparison_25years.png', dpi=200, bbox_inches='tight', facecolor='white')
plt.show()

print("\nFigure saved to ../figures/modflow_gwflow_comparison_25years.png")

In [ ]:
# =====================================================
# Cell 4: Year 25 Detail Comparison
# =====================================================

fig, ax = plt.subplots(figsize=(12, 6))
ax.set_facecolor('#e8e8ff')

months = ['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC']
x_months = np.arange(12)

# Plot EED reference (Year 25)
ax.plot(x_months, eed_base, 'ko--', lw=2, markersize=8, label='EED (reference)')

# Plot each scenario (Year 25)
for scenario in scenarios:
    if scenario in results:
        Tf_jan = np.array(results[scenario]['Tf_jan'])
        Tf_y25 = Tf_jan[-12:]  # Last year
        ax.plot(x_months, Tf_y25, 
                color=colors[scenario], marker='o', lw=2, markersize=6,
                label=f"{labels[scenario]}")

ax.set_xlabel('Month', fontsize=12)
ax.set_ylabel('Fluid temperature [°C]', fontsize=12)
ax.set_title('Year 25 Monthly Fluid Temperature Comparison', fontsize=14, fontweight='bold')
ax.set_xticks(x_months)
ax.set_xticklabels(months)
ax.set_ylim(9, 20)
ax.grid(True, alpha=0.3)
ax.legend(loc='upper right', fontsize=11)

plt.tight_layout()
plt.savefig('../figures/modflow_gwflow_comparison_year25.png', dpi=200, bbox_inches='tight', facecolor='white')
plt.show()

print("\nFigure saved to ../figures/modflow_gwflow_comparison_year25.png")

In [ ]:
# =====================================================
# Cell 5: Summary Statistics Table
# =====================================================

print("="*80)
print("GROUNDWATER FLOW SENSITIVITY ANALYSIS - SUMMARY")
print("="*80)

print(f"\n{'Scenario':<12} {'v [m/d]':<10} {'MAE [°C]':<12} {'Tf_min [°C]':<12} {'Tf_max [°C]':<12} {'Peak Heat':<12} {'Peak Cool':<12}")
print("-"*80)

for scenario in scenarios:
    if scenario in results:
        r = results[scenario]
        print(f"{scenario.upper():<12} {r['velocity_md']:<10.1f} {r['mae_vs_eed']:<12.3f} {r['Tf_min']:<12.2f} {r['Tf_max']:<12.2f} {r['peak_heat_min']:<12.2f} {r['peak_cool_max']:<12.2f}")
    else:
        print(f"{scenario.upper():<12} {'N/A':<10} {'N/A':<12} {'N/A':<12} {'N/A':<12} {'N/A':<12} {'N/A':<12}")

print(f"\n{'EED Ref':<12} {'0.0':<10} {'-':<12} {'10.5':<12} {'18.2':<12} {'6.91':<12} {'22.4':<12}")

In [ ]:
# =====================================================
# Cell 6: MAE Bar Chart
# =====================================================

fig, ax = plt.subplots(figsize=(8, 6))

available_scenarios = [s for s in scenarios if s in results]
mae_values = [results[s]['mae_vs_eed'] for s in available_scenarios]
bar_colors = [colors[s] for s in available_scenarios]
bar_labels = [f"v={velocities[s]} m/d" for s in available_scenarios]

bars = ax.bar(bar_labels, mae_values, color=bar_colors, edgecolor='black', linewidth=1.5)

# Add value labels on bars
for bar, mae in zip(bars, mae_values):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
            f'{mae:.3f}°C', ha='center', va='bottom', fontsize=11, fontweight='bold')

ax.set_ylabel('MAE vs EED [°C]', fontsize=12)
ax.set_title('Effect of Groundwater Velocity on Temperature Prediction\n(MAE compared to EED - no flow reference)', fontsize=12, fontweight='bold')
ax.set_ylim(0, max(mae_values) * 1.3)
ax.axhline(y=0.1, color='gray', ls='--', lw=1.5, label='Target MAE < 0.1°C')
ax.legend(loc='upper right')

plt.tight_layout()
plt.savefig('../figures/modflow_gwflow_mae_comparison.png', dpi=200, bbox_inches='tight', facecolor='white')
plt.show()

print("\nFigure saved to ../figures/modflow_gwflow_mae_comparison.png")

In [ ]:
# =====================================================
# Cell 7: Peak Temperature Analysis
# =====================================================

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

available_scenarios = [s for s in scenarios if s in results]

# Peak Heating (Winter minimum)
peak_heat = [results[s]['peak_heat_min'] for s in available_scenarios]
x_pos = np.arange(len(available_scenarios))
bars1 = ax1.bar(x_pos, peak_heat, color=[colors[s] for s in available_scenarios], edgecolor='black')
ax1.axhline(y=eed_peak_heat_jan, color='gray', ls='--', lw=2, label=f'EED: {eed_peak_heat_jan}°C')
ax1.set_xticks(x_pos)
ax1.set_xticklabels([f"v={velocities[s]}" for s in available_scenarios])
ax1.set_ylabel('Peak Heat Temperature [°C]', fontsize=11)
ax1.set_title('Winter Peak (JAN) - Heat Extraction', fontsize=12, fontweight='bold')
ax1.legend()
for bar, val in zip(bars1, peak_heat):
    ax1.text(bar.get_x() + bar.get_width()/2, val + 0.1, f'{val:.2f}', ha='center', fontsize=10)

# Peak Cooling (Summer maximum)
peak_cool = [results[s]['peak_cool_max'] for s in available_scenarios]
bars2 = ax2.bar(x_pos, peak_cool, color=[colors[s] for s in available_scenarios], edgecolor='black')
ax2.axhline(y=eed_peak_cool_aug, color='gray', ls='--', lw=2, label=f'EED: {eed_peak_cool_aug}°C')
ax2.set_xticks(x_pos)
ax2.set_xticklabels([f"v={velocities[s]}" for s in available_scenarios])
ax2.set_ylabel('Peak Cool Temperature [°C]', fontsize=11)
ax2.set_title('Summer Peak (AUG) - Heat Injection', fontsize=12, fontweight='bold')
ax2.legend()
for bar, val in zip(bars2, peak_cool):
    ax2.text(bar.get_x() + bar.get_width()/2, val + 0.1, f'{val:.2f}', ha='center', fontsize=10)

plt.tight_layout()
plt.savefig('../figures/modflow_gwflow_peak_comparison.png', dpi=200, bbox_inches='tight', facecolor='white')
plt.show()

print("\nFigure saved to ../figures/modflow_gwflow_peak_comparison.png")

In [ ]:
# =====================================================
# Cell 8: Year 25 Detailed Table
# =====================================================

months = ['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC']

print("="*90)
print("YEAR 25 MONTHLY FLUID TEMPERATURE COMPARISON [°C]")
print("="*90)

header = f"{'Month':<6} {'EED':>10}"
for scenario in scenarios:
    if scenario in results:
        header += f" {scenario.upper():>10}"
print(header)
print("-"*60)

for i, m in enumerate(months):
    row = f"{m:<6} {eed_base[i]:>10.2f}"
    for scenario in scenarios:
        if scenario in results:
            Tf_jan = np.array(results[scenario]['Tf_jan'])
            row += f" {Tf_jan[-12+i]:>10.2f}"
    print(row)

print("\nNote: Higher velocity → smaller temperature fluctuation (heat buffering effect)")